### Libraries Imports

In [1]:
import pandas as pd
import functools
import numpy as np

import cufflinks as cf
import plotly.offline
import plotly.graph_objs as go
import plotly.plotly as py
import seaborn as sns
import matplotlib.pylab as plt
import plotly.offline
import plotly.graph_objs as go
import plotly.plotly as py
import math
import calendar
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 500)
pd.set_option('max_colwidth', 5000)
pd.set_option('display.float_format', '{:,}'.format)
pd.options.display.float_format = '{:,.0f}'.format
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)
py.sign_in('Zawar', 'nyd3v1Zb2R1iXqHDUuhU')
pd.options.mode.chained_assignment = None
from datetime import datetime
day_of_year = datetime.now().timetuple().tm_yday

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

/home/data/.local/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning:

pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.



## Load Premium , Vehcile & Claims Data

In [283]:
#DONOT DELETE   READING FROM DATABASE 
from datetime import datetime 

start_time = datetime.now() 
print(start_time)

import time

start_date_2015 = ['2019-01-01']
end_date_2015 = ['2020-03-31']

import pyodbc

conn = pyodbc.connect("DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={0}; database={1};UID={2};PWD={3}".format('172.16.2.86\SQL14','TDIIMSDAILY','bigdata','BIG!@#data'))
cursor = conn.cursor()
tempDf1 = pd.DataFrame()
for f, b in zip(start_date_2015, end_date_2015):
    print(f,b)
    storedProc = "EXEC [CLA_MotorIntimateReport_Current_DS_SP] @Company_Id=4, @ClaimStartDateFrom='"+str(f)+"', @EndTime='"+str(b)+"' "
    dfCurrent = pd.read_sql_query(storedProc, conn)
    dfCurrent = pd.concat([dfCurrent,tempDf1])
    tempDf1 = dfCurrent

print('Time elapsed (hh:mm:ss.ms) {}'.format(datetime.now() - start_time))


2020-06-26 11:28:57.758922
2019-01-01 2020-03-31
Time elapsed (hh:mm:ss.ms) 0:15:33.776500


In [284]:
# import os 
# import pickle
# if os.path.exists('claimsRawData.pickle'):
#     with open('claimsRawData.pickle','rb') as f:
#         tempDf1 = pickle.load(f)
        
# df_claims = tempDf1.copy()
# df_claims.shape
# df_claims[df_claims.ClaimNo == 'ISB/P/UBL/C/176445/0819'][['GrossAmount', 'NetAmount','PaidTo','PaymentAmount','InvoiceAmount', 'WHTAmount',
#                                                          'Amount_Reserve', 'Reserve_GIS','RI_Amount', 'RI_Percent','CI_Amount', 'CI_Percent',
#                                                          'OutstandingPremium', 'SalvageRecovery', 'EstSalvage']]

import pickle
with open('claimsRawData.pickle', 'wb') as f:
    pickle.dump(df_claims, f)


In [3]:
#DONOT DELETE   READING FROM DATABASE 

from datetime import datetime 

start_time = datetime.now() 
print(start_time)

import time

start_date_2015 = ['2019-01-01']
end_date_2015 = ['2020-03-31']


import pyodbc

conn = pyodbc.connect("DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={0}; database={1};UID={2};PWD={3}".format('172.16.2.86\SQL14','TDIIMSDAILY','bigdata','BIG!@#data'))
cursor = conn.cursor()
tempDf2 = pd.DataFrame()
for f, b in zip(start_date_2015, end_date_2015):
    print(f, b)
    storedProc = "EXEC [SLS_GetVehicleWisePremiumRegister_DS_SP] @Company_Id1=4, @StartDate1='"+str(f)+"', @EndDate1='"+str(b)+"' , @Account_Type1=NULL "
    dfCurrent = pd.read_sql_query(storedProc, conn)
    dfCurrent = pd.concat([dfCurrent,tempDf2])
    tempDf2 = dfCurrent

print('Time elapsed (hh:mm:ss.ms) {}'.format(datetime.now() - start_time))

2020-07-27 15:57:36.153645
2019-01-01 2020-03-31
Time elapsed (hh:mm:ss.ms) 0:01:34.279003


In [286]:
# if os.path.exists('premRawData.pickle'):
#     with open('premRawData.pickle','rb') as f:
#         tempDf2 = pickle.load(f)

tempDf2.rename(columns={'branch': 'SalesBranch','PolicyNo':'SalesFormNo'}, inplace=True)
df_prem = tempDf2.copy()
df_prem.VoucherType = np.where( (df_prem.VoucherType.str.contains("NB") == True), "Fresh","Renewal")


df_prem.shape


with open('premRawData.pickle', 'wb') as f:
    pickle.dump(df_prem, f)

df_prem.VehicleGrossAmount.sum()



(74377, 84)

2614950291.0

## PreProcessing Data

In [287]:
df_claims['EffectiveDate'] =  pd.to_datetime(df_claims['EffectiveDate'],errors='coerce', format='%Y-%m-%d %H:%M:%S')
df_claims['IncidentDate'] =  pd.to_datetime(df_claims['IncidentDate'],errors='coerce', format='%Y-%m-%d %H:%M:%S')
df_claims['IncidentYear'] = df_claims['IncidentDate'].dt.year
df_claims['IntimationDate'] =  pd.to_datetime(df_claims['IntimationDate'],errors='coerce', format='%Y-%m-%d %H:%M:%S')
df_claims['IntimationYear'] = df_claims['IntimationDate'].dt.year


df_claims['VehicleMake'] = np.where(df_claims.VehicleMake.isin(['Suzuki','Toyota','Honda','Kia']) == False, 'Others', df_claims['VehicleMake'])

df_claims.shape
df_prem.shape

(24438, 83)

(74377, 84)

#### Premium Data

In [288]:
df_prem['BillDate'] =  pd.to_datetime(df_prem['BillDate'],errors='coerce', format='%Y-%m-%d %H:%M:%S')

df_prem['Year'] = df_prem['BillDate'].dt.year

df_prem['ExpiryDate'] =  pd.to_datetime(df_prem['ExpiryDate'],errors='coerce', format='%Y-%m-%d %H:%M:%S')

df_prem['EffectiveDate'] =  pd.to_datetime(df_prem['EffectiveDate'],errors='coerce', format='%Y-%m-%d %H:%M:%S')

df_prem['InsuredAmount'] = pd.to_numeric(df_prem['InsuredAmount'],errors='coerce')
df_prem['Adv_Tax'] = pd.to_numeric(df_prem['Adv_Tax'],errors='coerce')

df_prem.VehicleType = df_prem.VehicleType.fillna('None')
df_prem.shape
#df_prem.drop((df_prem[df_prem['EffectiveDate'].isnull()].index) | (df_prem[df_prem['ExpiryDate'].isnull()].index), inplace = True)



(74377, 85)

In [289]:
df_prem.head(1)

,Id,SalesformMapping_Id,row_number,groupid,Activity_Id,Cancellation_Type,SalesFormTaxesGroup_Id,Account_Type,CustomerType,Parent_SalesFormGroup_Id,Business,EndorsementNo,BillNo,BillDate,VoucherType,IsExistingCustomer,SalesFormNo,EffectiveDate,ExpiryDate,IssueDate,InsuredName,InsuredAmount,City,m_Registration_No,m_Engine_No,m_Chasis_No,VehicleMake,CommRate,CommAmount,IsLeadAgent,VehicleGrossAmount,ASCAmount,FEDAmount,FIFAmount,Adv_Tax,StampDuty,NetAmountT,NetRate,GrossRate,IntimationDate,ClaimNo,Reserve,SettleAmount,ClaimType,Showroom,LeadSource,InternalAgent,ParkingType,DriverType,m_CurrentMileage,Powered,Transmission_Type,CC,Color,Gender,CNIC,Address,Area,ContactCity,Occupation,HomeNo,OfficeNo,MobileNo,Status,AGE,SalesBranch,clpaid,clcount,CustomerSince,Product,PreRiskInspector,VehicleType,UW_Year,VehicleCategory,Outstanding,TransactionType_Id,Account_Number,Active/InActive,Account_Name,BasicPremium,LastActivity,Model,Make,ModelYear,Year
0,1432550,2015835,54,1,57,None,1290658,I,Individual,"602,531.00",NB,,AR-MTR-PRM-3098945,2019-01-01,Renewal,Yes,KHI/P/PC/37055/656953/0119,2019-01-04,2020-01-03,2019-01-01,M/S Shipco Transport Pakistan (Pvt.) Ltd User: Mr. Hussain,"1,337,000.00",Karachi,AVA-690,Y294831,2086287,Toyota Corolla XLI 1.3L 2011,2.00,669.00,Call Centre Commission (RENEWAL),"33,426.00","1,671.00","4,563.00",351.00,0.00,50.00,"40,011.00",2.99,2.50,2019-05-27 12:08:45.857,KHI/P/PC/C/171663/0519,0.00,848.00,Accident,NONE,Non 3S,Syeda Sadia Nizami,Bunglow,Self,"10,000.00",Patrol,Manual,1300,Black,Male,42301-5310377-9,"Suit# 702-703-704, 7th Floor, Progressive Plaza,\r\nBeaumont Road",Civil Lines,Karachi,-,None,None,None,Active,20.00,Karachi,848.00,1,10,Comprehensive Policy,None,Private Motor Car,2019,Local,0.00,4,37055,InActive,GENRAL MARITIME PVT LTD USER ADNAN BIN ISLAM,"31,426.00",Policy,Corolla XLI 1.3L,Toyota,2011,2019


In [290]:
print(df_prem.shape,df_claims.shape)
df_prem.BillDate.max()

(74377, 85) (24438, 83)


Timestamp('2020-03-31 21:49:27.960000')

#### Vehicle Data

In [153]:
def float_apply(x):
    try:
        obj = float(x)
        return True
    except ValueError:
        return False
    
def to_make_model(s):
    make  = str(s.split(" ",1)[0])
    model = s[len(make):-4]
    year  = s[-4:]
    return make, model, year

In [291]:
df_prem.dropna(subset=['VehicleMake'],inplace=True)

# df_prem['test'] = df_prem.VehicleMake.map(lambda x: to_make_model(x))

# new = pd.DataFrame(df_prem['test'].tolist(), index=df_prem.index)
# new.columns = ['Make','Model','CarYear']

# df_prem = pd.concat([df_prem, new], axis= 1)
# df_prem.drop('test', axis=1, inplace=True)
df_prem['Make'] = np.where(df_prem.Make.isin(['Suzuki','Toyota','Honda','Kia']) == False, 'Others', df_prem['Make'])

df_prem.shape

(74377, 85)

In [293]:
#df_prem[(df_prem.Make == 'Honda') & (df_prem['Model'].str.lower().str.contains("city|civic|civi|brv|br-v") == True)].VehicleCategory.value_counts()

#df_prem[(df_prem.Make == 'Toyota') & (df_prem.VehicleCategory == 'Local') ].Model.value_counts() 

#df_prem[df_prem.Model == 'TOYOTA COROLLA GLI 1.3 EFI'] #Gli ECOTEC , TOYOTA COROLLA GLI 1.3 EFI ,Toyota Corolla XLI LE


df_prem.VehicleCategory.value_counts()
df_prem.Make.value_counts()
df_prem.Model.value_counts()

#df_claims.VehicleType.value_counts()
#df_prem.VehicleMake.value_counts()

# df_prem[(df_prem.Make == 'Suzuki') & (df_prem.VehicleCategory != 'Imported')].VehicleGrossAmount.sum()
# df_prem[(df_prem.Make == 'Honda') & (df_prem.VehicleCategory != 'Imported')].VehicleGrossAmount.sum()
# df_prem[(df_prem.Make == 'Toyota') & (df_prem.VehicleCategory != 'Imported')].VehicleGrossAmount.sum()
# df_prem[(df_prem.VehicleCategory == 'Imported')].VehicleGrossAmount.sum()


Local       59262
Imported     9695
Others       5380
Name: VehicleCategory, dtype: int64

Toyota    25378
Suzuki    24730
Honda     15330
Others     8398
Kia         541
Name: Make, dtype: int64

Corolla GLI A/T    2872
Civic              2789
Wagon R VXL        2625
Corolla GLI        2587
City               2567
                   ... 
JADE HYBIRD           1
Charade               1
Margalla              1
FTR33K – PRM          1
Civic MTS             1
Name: Model, Length: 849, dtype: int64

In [294]:
df_prem[df_prem.VehicleCategory.isna()].Model.value_counts()

#df_prem[df_prem.VehicleCategory.isna()].VehicleGrossAmount.sum()#.value_counts()



df_claims[df_claims.VehicleType.isna()].VehicleModel.value_counts()#.value_counts()


Rickshaw    35
N-Wagon      5
Name: Model, dtype: int64

N-Wagon    3
Name: VehicleModel, dtype: int64

In [295]:

# df_prem.VehicleCategory = 'Empty'

# df_prem.VehicleCategory =  np.where((df_prem.Make == 'Honda') & (df_prem['Model'].str.lower().str.contains("city|civic|civi|brv|br-v") == True), 'Local', df_prem.VehicleCategory)

# df_prem.VehicleCategory =  np.where((df_prem.Make == 'Toyota') & (df_prem['Model'].str.lower().str.contains("vigo|revo|fortun|corolla") == True), 'Local', df_prem.VehicleCategory)

# df_prem.VehicleCategory =  np.where((df_prem.Make == 'Kia') & (df_prem['Model'].str.lower().str.contains("carnival|picanto|sportage") == True), 'Local', df_prem.VehicleCategory)

# df_prem.VehicleCategory =  np.where((df_prem.Make == 'Suzuki') & (df_prem['Model'].str.lower().str.contains("apv|baleno|bolan|ciaz|cultus|khyber|kizashi|liana|margalla|mehran|swift|wagon|alto") == True), 'Local', df_prem.VehicleCategory)

# df_prem.VehicleCategory =  np.where((df_prem['Model'].str.lower().str.contains("mp 900|m 280|power cd|rickshaw|d-max|coure|charade|bravo|jeep|lancer|pajero|potohar|prince|santro|sirus|subaru|sunny|v2|van|ravi") == True), 'Others', df_prem.VehicleCategory)

# df_prem.VehicleCategory =  np.where((df_prem['VehicleType'].str.lower().str.contains("commercial|motorcycle|none") == True), 'Others', df_prem.VehicleCategory)


df_prem.VehicleCategory =  np.where((df_prem.Make == 'Others') & (df_prem['Model'].str.lower().str.contains("moov|mira|dazey|n-wagon") == True), 'Imported', df_prem.VehicleCategory)
df_prem.VehicleCategory =  np.where((df_prem.Make == 'Others') & (df_prem['Model'].str.lower().str.contains("power|rickshaw") == True), 'Others', df_prem.VehicleCategory)

df_prem['Make'] = np.where((df_prem.VehicleCategory.isna() == True), 'Others', df_prem.Make)

df_prem.VehicleCategory =  np.where((df_prem.VehicleCategory.isna() == True), 'Others', df_prem.VehicleCategory)

df_prem['Make'] = np.where(df_prem.VehicleCategory == 'Imported', 'Imported', df_prem.Make)
#df_prem['Make'] = np.where(df_prem.VehicleCategory == 'Others', 'Others', df_prem.Make)

In [296]:

# df_claims.VehicleType = 'Empty'

# df_claims.VehicleType =  np.where((df_claims.VehicleMake == 'Honda') & (df_claims['VehicleModel'].str.lower().str.contains("city|civic|civi|brv|br-v") == True), 'Local', df_claims.VehicleType)

# df_claims.VehicleType =  np.where((df_claims.VehicleMake == 'Toyota') & (df_claims['VehicleModel'].str.lower().str.contains("vigo|revo|fortun|corolla") == True), 'Local', df_claims.VehicleType)

# df_claims.VehicleType =  np.where((df_claims.VehicleMake == 'Kia') & (df_claims['VehicleModel'].str.lower().str.contains("carnival|picanto|sportage") == True), 'Local', df_claims.VehicleType)

# df_claims.VehicleType =  np.where((df_claims.VehicleMake == 'Suzuki') & (df_claims['VehicleModel'].str.lower().str.contains("apv|baleno|bolan|ciaz|cultus|khyber|kizashi|liana|margalla|mehran|swift|wagon|alto") == True), 'Local', df_claims.VehicleType)

# df_claims.VehicleType =  np.where((df_claims['VehicleModel'].str.lower().str.contains("mp 900|m 280|power cd|rickshaw|d-max|coure|charade|bravo|jeep|lancer|pajero|potohar|prince|santro|sirus|subaru|sunny|v2|van|rabvi") == True), 'Others', df_claims.VehicleType)

# df_claims.VehicleType =  np.where((df_claims['VehicleCategory'].str.lower().str.contains("commercial|motorcycle|none") == True), 'Others', df_claims.VehicleType)

# df_claims.VehicleType =  np.where((df_claims.VehicleType == 'Empty'), 'Imported', df_claims.VehicleType)

df_claims.VehicleType =  np.where((df_claims.VehicleMake == 'Others') & (df_claims['VehicleModel'].str.lower().str.contains("moov|mira|dazey|n-wagon") == True), 'Imported', df_claims.VehicleType)
df_claims.VehicleType =  np.where((df_claims.VehicleMake == 'Others') & (df_claims['VehicleModel'].str.lower().str.contains("power|rickshaw") == True), 'Others', df_claims.VehicleType)

df_claims['VehicleMake'] = np.where((df_claims.VehicleType.isna() == True), 'Others', df_claims.VehicleMake)
#df_claims.VehicleType =  np.where((df_claims.VehicleType.isna() == True), 'Others', df_claims.VehicleType)

df_claims['VehicleMake'] = np.where(df_claims.VehicleType == 'Imported', 'Imported', df_claims.VehicleMake)
#df_claims['VehicleMake'] = np.where(df_claims.VehicleType == 'Others', 'Others', df_claims.VehicleMake)
#df_claims.VehicleType.value_counts()

In [297]:
df_claims.VehicleMake.value_counts()
#df_claims[(df_claims.VehicleMake == 'Honda') & (df_claims.VehicleType == 'Local')].PaidTo.sum()
#df_claims.VehicleType.value_counts()
df_claims.shape

Suzuki      8147
Toyota      7000
Honda       5768
Imported    3066
Others       442
Kia           15
Name: VehicleMake, dtype: int64

(24438, 83)

#### Claims Data

#### Columns Selection

In [298]:
df_prem[df_prem.VehicleCategory.isna()].Model.value_counts()
df_claims[df_claims.VehicleType.isna()].VehicleModel.value_counts()
df_claims.VehicleType.value_counts()
df_prem.VehicleCategory.value_counts()
#tt[tt.duplicated(['SalesFormNo'], keep=False)].to_csv('duplicate.csv', index = False, header=True)
#df_claims[['ClaimNo','PaidTo','PaymentAmount','CI_Amount','SalvageRecovery','VehicleMake']].to_csv('claims_test.csv', index = False, header=True)
#df_prem.VehicleCategory.isna().any()
"N-Wagon".lower()

Series([], Name: Model, dtype: int64)

Series([], Name: VehicleModel, dtype: int64)

Local       21007
Imported     3066
Others        365
Name: VehicleType, dtype: int64

Local       59259
Imported     9695
Others       5423
Name: VehicleCategory, dtype: int64

'n-wagon'

In [299]:
df_prem['Policy Count'] = df_prem.groupby(['Make',
                                           'Model',
                                           'Year',
                                           'VoucherType',
                                           'SalesBranch',
                                           'VehicleCategory',
                                           'LeadSource'])['SalesFormNo'].transform('nunique')



premCols = [
         'SalesFormNo',
         'Year',
         'Product',
         'Make',
    'Model',
         'VoucherType',
         'SalesBranch',
         'Account_Name',
         'Account_Number',
         'LeadSource',
         'VehicleCategory',
         'VehicleGrossAmount',
         'Policy Count',
         
]

groupbyCols = [
               'Year',
               'Make',
    'Model',
               'VoucherType',
               'SalesBranch',
               'LeadSource',
               'VehicleCategory',
               'Policy Count',
               
              ]



dfp_grp = df_prem#[premCols]
dfp_grp = dfp_grp.groupby(groupbyCols)['VehicleGrossAmount'].agg(['sum']).reset_index()
dfp_grp.columns = dfp_grp.columns.get_level_values(0)

dfp_grp['Model'] = dfp_grp.Model.str.strip()

dfp_grp.rename(columns={'branch': 'SalesBranch','VoucherType':'BusinessType','LeadSource':'Channel','sum':'Gross Premium','count':'Policy Count'}, inplace=True)


dfp_grp['Gross Premium'].sum()
#df_prem.VehicleGrossAmount.sum()
dfp_grp['Policy Count'].sum()
dfp_grp
   
#dfp_grp[(dfp_grp.SalesBranch == 'Islamabad')  & (dfp_grp.Channel == '3S') & (dfp_grp.BusinessType == 'Fresh') & (dfp_grp.Make == 'Honda') & (dfp_grp.VehicleCategory == 'Imported')]


2614950291.0

63082

,Year,Make,Model,BusinessType,SalesBranch,Channel,VehicleCategory,Policy Count,Gross Premium
0,2019,Honda,BR-V,Fresh,Faisalabad,3S,Local,11,"606,381.00"
1,2019,Honda,BR-V,Fresh,Faisalabad,Direct,Local,4,"278,048.00"
2,2019,Honda,BR-V,Fresh,Faisalabad,Non 3S,Local,1,"67,691.00"
3,2019,Honda,BR-V,Fresh,Gujranwala,Direct,Local,1,"99,923.00"
4,2019,Honda,BR-V,Fresh,Gujranwala,Non 3S,Local,1,"72,800.00"
...,...,...,...,...,...,...,...,...,...
8807,2020,Toyota,Toyota Corolla XLI,Renewal,Karachi,Non 3S,Local,1,"21,079.00"
8808,2020,Toyota,Xli,Fresh,Karachi,CallCentre,Local,1,"60,675.00"
8809,2020,Toyota,Xli,Renewal,Lahore,CallCentre,Local,2,"105,827.00"
8810,2020,Toyota,corolla 1.6 Gli A/T,Renewal,Karachi,AD,Local,1,"39,014.00"


In [300]:
def trim_all_columns(df):
    """
    Trim whitespace from ends of each value across all series in dataframe
    """
    trim_strings = lambda x: x.strip() if isinstance(x, str) else x
    return df.applymap(trim_strings)

#prem_bk = df_prem.copy()
#prem_bk[['SalesFormNo','Make','Year','VoucherType','SalesBranch','VehicleCategory','LeadSource']] = prem_bk[['SalesFormNo','Make','Year','VoucherType','SalesBranch','VehicleCategory','LeadSource']].astype(str)
#prem_bk = trim_all_columns(prem_bk)
#duplicats = prem_bk.groupby(['SalesFormNo','Make'])['SalesFormNo'].size()
#duplicats[duplicats > 1]


#### Join Prem & Vehcile Data

In [301]:
#df_claims.fillna(0,inplace=True) 
#df_claims['VehicleMake'] = np.where(df_claims.VehicleMake.isin(['Suzuki','Toyota','Honda','Kia']) == False, 'Others', df_claims['VehicleMake'])    
#df_claims.VehicleMake.unique()
#df_claims[df_claims.VehicleType != 0]

###FILL NA VEHICLE TYPE TO OTHERS

In [302]:
df_claims.VehicleType.isna().any()
df_prem.SalesFormNo.nunique()

False

60915

In [303]:
df_claims[df_claims.IntimationYear == 2019].shape


(24438, 83)

#### Join Prem,Vehicle & Claim Amount Data

In [304]:
#SalesFormNo',
df_claims.Amount_Reserve.fillna(0,inplace=True) #AG
df_claims.PaidTo.fillna(0,inplace=True) #AH

df_claims.SalvageRecovery.fillna(0,inplace=True) # AL
df_claims.EstSalvage.fillna(0,inplace=True)

df_claims.PaymentAmount.fillna(0,inplace=True) #AN

df_claims.CI_Amount.fillna(0,inplace=True) # AM
df_claims.CI_Percent.fillna(0,inplace=True) #W

df_claims.CI_Amount = (df_claims.Amount_Reserve + df_claims.PaidTo) * (df_claims.CI_Percent/100)

claimCols = [
             'IntimationYear',
             'PaidTo',
             'SalvageRecovery',
             'VehicleMake',
    'VehicleModel',
             'ClaimTypes',
             'BusinessType',
             'BranchName',
             'LeadSource',
             'VehicleType',
             'CI_Amount',
             'Amount_Reserve',   
             'ClaimNo',
             'EstSalvage'
         
]

groupbyCols = [
               'IntimationYear',
               'VehicleMake',
    'VehicleModel',
                 'ClaimTypes',
                 'BusinessType',
                 'BranchName',
                 'LeadSource',
                 'VehicleType',
                 'PaidTo',
                 'SalvageRecovery',
                 'CI_Amount',
                 'Amount_Reserve', 
                 'ClaimNo',
                 'EstSalvage'
               
              ]

#df_claims[(df_claims.IntimationYear == 2019) & (df_claims.ClaimStage != 'ClaimClose')].PaidTo.sum() - df_claims[(df_claims.IntimationYear == 2019) & (df_claims.ClaimStage != 'ClaimClose')].SalvageRecovery.sum()

df_claims['VehicleType'] = np.where(df_claims.VehicleType == 0, 'Others', df_claims.VehicleType)
#df_claims['VehicleMake'] = np.where(df_claims.VehicleType == 'Imported', 'Imported', df_claims.VehicleMake)

dfc_grp = df_claims[claimCols]#[df_claims.ClaimStage != 'ClaimClose'][claimCols]

dfc_grp = dfc_grp.groupby(groupbyCols)['PaidTo'].agg(['sum','count']).reset_index()
dfc_grp['sum'] = (dfc_grp.Amount_Reserve + dfc_grp.PaidTo) - (dfc_grp.SalvageRecovery + dfc_grp.EstSalvage) - dfc_grp.CI_Amount
dfc_grp.rename(columns={'BusinessType':'VoucherType','BranchName':'SalesBranch'}, inplace=True)
dfc_grp['ClaimTypes'] = np.where(dfc_grp.ClaimTypes.str.strip() == 'Snatch', 'Theft', dfc_grp.ClaimTypes)
dfc_grp['ClaimTypes'] = dfc_grp['ClaimTypes'].str.strip()

dfc_grp['VehicleModel'] = dfc_grp.VehicleModel.str.strip()

dfc_grp.rename(columns={'IntimationYear':'Year', 'VehicleMake':'Make','VehicleModel':'Model', 'VehicleType': 'VehicleCategory','VoucherType':'BusinessType','LeadSource':'Channel','sum':'TotalClaimed','count':'Claim Count'}, inplace=True)
dfc_grp[dfc_grp.Year == 2019].TotalClaimed.sum()
dfc_grp['Claim Count'].sum()
dfc_grp#[dfc_grp.Year == 2019]
#dfc_grp[(dfc_grp.SalesBranch == 'Karachi')  & (dfc_grp.Channel == '3S') & (dfc_grp.BusinessType == 'Fresh') & (dfc_grp.Make == 'Others') & (dfc_grp.VehicleCategory == 'Imported')]



946227784.8989999

24438

,Year,Make,Model,ClaimTypes,BusinessType,SalesBranch,Channel,VehicleCategory,PaidTo,SalvageRecovery,CI_Amount,Amount_Reserve,ClaimNo,EstSalvage,TotalClaimed,Claim Count
0,2019,Honda,BR-V,Accessory Theft,Fresh,Hyderabad,3S,Local,"6,310.00",0.00,0.00,0.00,HYD/P/WT/C/168866/0419,0.00,"6,310.00",1
1,2019,Honda,BR-V,Accessory Theft,Fresh,Karachi,3S,Local,"62,500.00",0.00,0.00,0.00,KHI/P/WT/C/173385/0619,0.00,"62,500.00",1
2,2019,Honda,BR-V,Accessory Theft,Fresh,Karachi,3S,Local,"85,440.00",0.00,0.00,0.00,KHI/P/WT/C/168712/0419,0.00,"85,440.00",1
3,2019,Honda,BR-V,Accessory Theft,Fresh,Lahore,3S,Local,0.00,0.00,0.00,0.00,LHR/P/WT/C/163782/0219,0.00,0.00,1
4,2019,Honda,BR-V,Accessory Theft,Fresh,Lahore,3S,Local,0.00,0.00,0.00,0.00,LHR/P/WT/C/164926/0219,0.00,0.00,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24433,2019,Toyota,corolla 1.6 Gli A/T,Accident,Renewal,Lahore,CallCentre,Local,"12,776.00",0.00,0.00,0.00,LHR/P/PC/C/181924/1019,0.00,"12,776.00",1
24434,2019,Toyota,corolla 1.6 Gli A/T,Accident,Renewal,Lahore,Non 3S,Local,"18,991.00",0.00,0.00,0.00,LHR/P/WT/C/184098/1219,0.00,"18,991.00",1
24435,2019,Toyota,corolla 1.6 Gli A/T,Accident,Renewal,Lahore,Non 3S,Local,"25,309.00",0.00,0.00,0.00,LHR/P/ABB/C/165106/0219,0.00,"25,309.00",1
24436,2019,Toyota,corolla 1.6 Gli A/T,Accident,Renewal,Lahore,Non 3S,Local,"27,608.00",0.00,0.00,0.00,LHR/P/PC/C/174868/0719,0.00,"27,608.00",1


In [305]:
tdf_prem = dfp_grp.groupby(['Year','Make','Model'])['Gross Premium','Policy Count'].agg(['sum']).reset_index()
tdf_claim = dfc_grp.groupby(['Year','Make','Model'])['TotalClaimed','Claim Count'].agg(['sum']).reset_index()

tdf_prem.columns = tdf_prem.columns.get_level_values(0)
tdf_claim.columns = tdf_claim.columns.get_level_values(0)
tdf_prem['Gross Premium'].sum()

tdf_claim.TotalClaimed.sum()
#tdf_prem
#tdf_claim
tdf = tdf_prem.merge(tdf_claim,on=['Year','Make'],how="left")
#tdf 

tdf_prem[tdf_prem.Make == 'Honda'].merge(tdf_claim[tdf_claim.Make == 'Honda'], on=['Make','Model'],how='left')
#tdf_prem[(tdf_prem.Make == 'Honda') & (tdf_prem.Model.str.strip() == 'BR-V')]
# tdf_prem.dtypes
# tdf_claim.dtypes

/home/data/anaconda3/envs/TestEnv/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.

/home/data/anaconda3/envs/TestEnv/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



2614950291.0

946227784.8989999

,Year_x,Make,Model,Gross Premium,Policy Count,Year_y,TotalClaimed,Claim Count
0,2019,Honda,BR-V,"38,766,912.00",751,"2,019.00","16,188,246.25",370.00
1,2019,Honda,BRV I-VTEC CVT (S),"98,610.00",2,nan,nan,nan
2,2019,Honda,BRV I-VTEC M/T,"3,799,975.00",71,"2,019.00","1,037,835.00",24.00
3,2019,Honda,BRV IVTEC CVT,"602,748.00",14,"2,019.00","337,909.00",3.00
4,2019,Honda,BRV-IVS,"928,049.00",19,"2,019.00","461,747.00",15.00
5,2019,Honda,BRV-S,"2,330,471.00",39,"2,019.00","200,987.00",3.00
6,2019,Honda,CDI,"5,809.00",5,nan,nan,nan
7,2019,Honda,CG-125,"63,398.00",9,nan,nan,nan
8,2019,Honda,CITY 1.3 MANUAL TRANSMISSION,"2,162,113.00",68,"2,019.00","771,353.00",45.00
9,2019,Honda,CITY 1.3 VARIO,"19,674.00",1,nan,nan,nan


In [307]:
import humanize as hm
print('Data from 2019 to 2020 Feb 27th')
dict4Model = {}

# dict4Model['df_prem'] = df_premdf_prem
# dict4Model['df_vehicle'] = df_vehicle
# dict4Model['dfc_grp'] = dfc_grp

# dict4Model['data'] = tdf

businessTypeList = dfp_grp.BusinessType.unique().tolist()
businessTypeList.insert(0, 'All')
# dict4Model['BusinessTypes'] = businessTypeList

branchList = dfp_grp.SalesBranch.unique().tolist()
branchList.insert(0, 'All')
# dict4Model['Branches'] = branchList

claimTypeList = dfc_grp.ClaimTypes.str.strip().unique().tolist()
claimTypeList.insert(0, 'All')
# dict4Model['ClaimTypes'] = claimTypeList

# claimStageList = dfc_grp.ClaimStage.str.strip().unique().tolist()
# claimStageList.insert(0, 'All')
# dict4Model['ClaimTypes'] = claimTypeList

channelList = dfp_grp.Channel.str.strip().unique().tolist()
channelList.insert(0, 'All')
# dict4Model['ChannelTypes'] = channelList

vehicleMakeList = dfp_grp.Make.str.strip().unique().tolist()
vehicleMakeList.insert(0, 'All')

yearList = dfp_grp.Year.unique().tolist()
yearList.insert(2, 'All')

# import pickle
# with open('/home/data/Projects/MISLossProfile/pickle/modelData.pickle', 'wb') as f:
#     pickle.dump(dict4Model, f)

def MISReport(BusinessType,SalesBranch,Channel,Make,Year,ClaimTypes,High_Claim_Details,Misc_Report,premData,claimData):
    
    if (High_Claim_Details == False) & (Misc_Report == False):
        
        dataKVP = {'BusinessType': BusinessType, 'SalesBranch': SalesBranch, 'Channel': Channel, 'Year':Year, 'Make': Make, 'ClaimTypes':ClaimTypes}
        
        filter_str = buildFilter(dataKVP)
        
        if filter_str != -1:  ##if any filter applied 
            
            if "ClaimTypes" in filter_str: ## if ClaimFilter then dont use with PremData
                if len(filter_str.rsplit('and',1)) > 1: ## if len is 2 means other filters also selected with claimType
                    dfp_grp = premData.query(filter_str.rsplit('and',1)[0])
                    dfc_grp = claimData.query(filter_str)
                else:
                    dfp_grp = premData
                    dfc_grp = claimData.query(filter_str)
            else:
                dfp_grp = premData.query(filter_str)
                dfc_grp = claimData.query(filter_str)
        else:
            dfp_grp = premData
            dfc_grp = claimData
        
        DefaultGroupByList = ['Year','Make']
        
        if Make != 'All':
            print('Selected Makes/Models')
            DefaultGroupByList.extend(['Model'])
            tdf_prem = dfp_grp.groupby(DefaultGroupByList)['Gross Premium','Policy Count'].agg(['sum']).reset_index()
            tdf_claim = dfc_grp.groupby(DefaultGroupByList)['TotalClaimed','Claim Count'].agg(['sum']).reset_index()

        else:   
            tdf_prem = dfp_grp.groupby(DefaultGroupByList)['Gross Premium','Policy Count'].agg(['sum']).reset_index()
            tdf_claim = dfc_grp.groupby(DefaultGroupByList)['TotalClaimed','Claim Count'].agg(['sum']).reset_index()

        tdf_prem.columns = tdf_prem.columns.get_level_values(0)
        tdf_claim.columns = tdf_claim.columns.get_level_values(0)
        
        if Make != 'All':
            tdf = tdf_prem.merge(tdf_claim.drop_duplicates(),on=['Year','Make','Model'],how="left")
            tdf = pd.pivot_table(tdf,index=['Make','Model'],values=[ 'Gross Premium','Policy Count','TotalClaimed','Claim Count'],aggfunc=np.sum, margins=True)

        else:
            tdf = tdf_prem.merge(tdf_claim.drop_duplicates(),on=['Year','Make'],how="left")
            tdf = pd.pivot_table(tdf,index='Make',values=[ 'Gross Premium','Policy Count','TotalClaimed','Claim Count'],aggfunc=np.sum, margins=True)


        
        
        pd.options.display.float_format = '{:,.2f}'.format

        #tdf=tdf.stack(level=1)
    
        if tdf.shape[0] > 0:
            tdf['Claim Ratio%'] = (tdf['TotalClaimed'].astype(float) / tdf['Gross Premium'].astype(float)) * 100
            tdf['Claim Freq%'] = (tdf['Claim Count'].astype(float) / tdf['Policy Count'].astype(float)) * 100
        else:
            print('No Records Found')
            return None

        #tdf=tdf.unstack()
        #tdf.reorder_levels([1,0],axis=1).sort_index(axis=1)
        tdf.fillna(0,inplace=True)
       
        tdf = tdf.astype(int)
        #tdf = tdf.applymap(hm.intword)
        return tdf
    elif High_Claim_Details == True :
        return dfc_grp[(dfc_grp.ClaimAmount >= 500000)][dfc_grp.columns.difference(['CircumstancesOfLoss'])].applymap(hm.intword)
    else : 
        
        tdf = pd.read_csv('~/Projects/InsuranceAnalysis/data/CLA_MotorPaidReport_2018Onwards.csv',sep='|',engine='python')
        tdf['RecieptDate'] =  pd.to_datetime(tdf['RecieptDate'],errors='coerce', format='%Y-%m-%d %H:%M:%S')
        tdf = tdf[tdf.ClaimStage == 'Final Cost Estimation'].groupby([tdf.RecieptDate.dt.year,'PaymentHead'])['PaidAmount'].agg(['sum','count']).reset_index()
        tdf.columns = ['RecieptDate','PaymentHead', 'Calim Sum','No of Claims']
        tdf = tdf.pivot(index='PaymentHead',columns='RecieptDate',values=[ 'No of Claims','Claim Sum'])
        tdf = tdf.stack(level=1)
        tdf = tdf.astype(int)
        tdf = tdf.applymap(hm.intword)
        
        return tdf

    
def buildFilter(dataKVP):
    s = ''
    
    for k,v in dataKVP.items():
        if v != 'All':
            s += k + '==' + f'"{v}"' + ' and '
    
    if ('and' in s):
        print(s[:s.rindex('and')])
        return s[:s.rindex('and')]
    else:
        return -1
    
interact(MISReport,
         BusinessType=businessTypeList,
         SalesBranch=branchList,
         ClaimTypes=claimTypeList,
         Channel=channelList,
         Make=vehicleMakeList,
         Year=yearList,
         High_Claim_Details=False,Misc_Report=False,
         premData=fixed(dfp_grp),claimData=fixed(dfc_grp))

Data from 2019 to 2020 Feb 27th


interactive(children=(Dropdown(description='BusinessType', options=('All', 'Fresh', 'Renewal'), value='All'), …

<function __main__.MISReport(BusinessType, SalesBranch, Channel, Make, Year, ClaimTypes, High_Claim_Details, Misc_Report, premData, claimData)>

In [232]:
from datetime import datetime
from datetime import timedelta
today = datetime.today()
yesterday = today - timedelta(days=364 + datetime.now().timetuple().tm_yday)
print(today.now().date())
print()
print(yesterday.date())


2020-03-04

2019-01-01


In [113]:
# start_date = ['2020-01-01']
# end_date = ['2020-01-30']

# conn = pyodbc.connect("DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={0}; database={1};UID={2};PWD={3}".format('172.16.2.86\SQL14','TDIIMSDAILY','bigdata','BIG!@#data'))
# dfCurrentp = pd.DataFrame()
# tempDfp = pd.DataFrame()
# for f, b in zip(start_date, end_date):
#     #st.info('ClaimsVendorData From :' + str(f) + " to " + str(b))
#     print('ClaimsVendorData From :' + str(f) + " to " + str(b))
#     storedProc = "EXEC [CLA_MotorPaidReport_SP_New] @Company_Id=4, @ClaimStartDateFrom='"+str(f)+"', @ClaimStartDateTo='"+str(b)+"'"
#     dfCurrentp = pd.read_sql_query(storedProc, conn)
#     dfCurrentp = pd.concat([dfCurrentp,tempDfp])
#     tempDfp = dfCurrentp

tdf = tempDfp
#[tdf.ClaimStage == 'ClaimClose']
tdf['RecieptDate'] =  pd.to_datetime(tdf['RecieptDate'],errors='coerce', format='%Y-%m-%d %H:%M:%S')
tdf = tdf[tdf.ClaimStage != 'ClaimClose'].groupby([tdf.RecieptDate.dt.year,'PaymentHead'])['PaidAmount'].agg(['sum','count']).reset_index()
tdf.columns = ['RecieptDate','PaymentHead', 'Claim Sum','No of Claims']
tdf = tdf.pivot(index='PaymentHead',columns='RecieptDate',values=[ 'No of Claims','Claim Sum'])
tdf = tdf.stack(level=1)
tdf = tdf.astype(int)
tdf = tdf.applymap(hm.intword)

tdf

,,No of Claims,Claim Sum
PaymentHead,RecieptDate,,
INSURED,2020,137,47.3 million
RecoveryAgent,2020,2,130.0 thousand
Supplier,2020,84,1.4 million
Surveyor,2020,540,1.0 million
TowingCompany,2020,5,63.0 thousand
Workshop,2020,938,32.7 million
